# Global Setup

In [1]:
!pip install transformers
!pip install sentencepiece
!pip install segtok
!pip install vaderSentiment

In [2]:
import os
import sys
from pathlib import Path

import json
import pandas as pd
import random

import torch
from segtok import tokenizer
from keras.preprocessing.sequence import pad_sequences
import tqdm

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
ROOT_FOLDER = Path("/content/drive/My Drive/cs182_final_project/cs182-nlp")
DATA_FOLDER = ROOT_FOLDER / "dataset"

In [5]:
# sys.path.insert(0, "/content/drive/My Drive/cs182_final_project/cs182-nlp")
sys.path.insert(0, ROOT_FOLDER)
# sys.path.append(ROOT_FOLDER)

In [6]:
# !cp drive/MyDrive/cs182_final_project/cs182-nlp/data.py
# sys.path.append(ROOT_FOLDER/"data.py")
# sys.path.append(ROOT_FOLDER/"models.py")
from data import *
from models import LanguageModel

ModuleNotFoundError: ignored

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [ ]:
list_to_device = lambda th_obj: [tensor.to(device) for tensor in th_obj]

# Model Params

In [ ]:
MAX_LEN = 256
BATCH_SIZE = 16 # not used

#Data Preprocessing Functions

## load data

## data formatting

### tokenize

### padding

### batching

### full data format

### split dataset

# Data Preprocessing Code

## load data

In [ ]:
# load yelp data
yelp_reviews = load_json(DATA_FOLDER / "yelp_review_training_dataset.jsonl")
print("loaded", len(yelp_reviews.index), "data points")

loaded 533581 data points


In [ ]:
display(yelp_reviews)

,review_id,text,stars
0,Q1sbwvVQXV2734tPgoKj4Q,Total bill for this horrible service? Over $8G...,1.0
1,GJXCdrto3ASJOqKeVWPi6Q,I *adore* Travis at the Hard Rock's new Kelly ...,5.0
2,2TzJjDVDEuAW6MR5Vuc1ug,I have to say that this office really has it t...,5.0
3,yi0R0Ugj_xUx_Nek0-_Qig,Went in for a lunch. Steak sandwich was delici...,5.0
4,11a8sVPMUFtaC7_ABRkmtw,Today was my second out of three sessions I ha...,1.0
...,...,...,...
533576,2vQO_kmSr6YPBrR8GH_FPA,Dr Young and her assistants take obvious pride...,5.0
533577,DUdLTGVpgsi0sv_g4A5ITQ,We started our 20 month of daughter here on an...,5.0
533578,AKGELpRNTTXajuZHbPxdJg,"First of all, they are supposed to open at 9:0...",2.0
533579,ghYZM7lqzjej05I_T3vYyA,It's not often that you visit a company and th...,5.0


## format + split data into train, val, and test sets

In [ ]:
xlnet_tokenizer = torch.hub.load('huggingface/pytorch-transformers', 'tokenizer', 'xlnet-base-cased')
# tokenize_review(xlnet_tokenizer, "I love this grub!")

Using cache found in /root/.cache/torch/hub/huggingface_pytorch-transformers_master


In [ ]:
# train 75% | validation 15% | test 10%
train_ratio = .75
validate_ratio = .15
test_ratio = .10
assert train_ratio + validate_ratio + test_ratio == 1

In [ ]:
train_reviews, validate_reviews, test_reviews = train_validate_test_split(yelp_reviews, train_ratio, validate_ratio)
# train_reviews_df, val_reviews_df, test_reviews_df = train_validate_test_split(yelp_reviews, train_ratio, validate_ratio)

In [ ]:
# train_reviews, train_reviews_target, train_reviews_mask = format_reviews(xlnet_tokenizer, train_reviews_df)
# validate_reviews, test_reviews_target, validate_reviews_mask = format_reviews(xlnet_tokenizer, validate_reviews_df)
# test_reviews, test_reviews_target, _ = format_reviews(xlnet_tokenizer, test_reviews_df)

In [ ]:
print(len(train_reviews.index), "yelp reviews for training")
train_reviews

400185 yelp reviews for training


,review_id,text,stars
134046,uozx1JGyRjHzs_aASyPcXg,I had a bad experience with shitty ATTITUDE. B...,1.0
41553,Ahq1v_fyEy00giS3JC7aLg,I thought the food there was good for the pric...,3.0
225115,g4Uj59yJl118-rjlz22s0g,I don't know what happened to the other review...,5.0
17163,WgTjvFVihdfENQCym1ZMsg,Been a patient for 2 1/2 years. Be prepared to...,2.0
11902,aF79-cK0Gj-jEmmTkBmN4Q,I don't often write reviews but I was so impre...,5.0
...,...,...,...
385507,lmr-n6jfHYwEYOGYUY_8AQ,I went to a different office prior to EyesWest...,5.0
426778,w02Yb3hs-6zFCcbVLMD4GQ,Fab Restaurant Concepts Inc. recently launched...,4.0
410697,daRP_YY8SPhpd9pvPNPh8A,A Yeti Nails on college? Yes!!\n\nI always go ...,5.0
382385,ftzEU-izq2fAf6cysqqlqA,Well I've been with the PMG for years and I've...,1.0


In [ ]:
print(len(validate_reviews.index), "yelp reviews for validation")
validate_reviews

80037 yelp reviews for validation


,review_id,text,stars
293393,g4tyji9AbGA8s8d8wR1Ghw,Dr. Brown is an exceptional Doctor and an even...,5.0
91236,yv-QVVz8r9L0KzPE7dC9Kw,Very satisfied with them and they do deserve 5...,5.0
519480,u7ob6oGBXoxNmoNdO6Hi7A,Discovered this place when checking out The Do...,5.0
85922,D6_cubDDmUHK9etIo4qj5g,"good deals, okay pizza, good wings mango great...",5.0
272029,UT-44vSkncrfr7pTNV7Z2Q,We had to stay here for 2 weeks as our house w...,3.0
...,...,...,...
395020,PcpbnlwESsc2INBtySjNoQ,Wow!!!! New opening and the Staff is AWESOME!!...,5.0
368659,5suyUogE1SECWmNO4Od_jQ,Nice higher end steak and seafood. Have had m...,5.0
304554,VcZhY420gOcRvwC5Z6RyFw,I like mucho. But the portions here were ridic...,1.0
314880,qoeh99DeddHIX2cnFC5C5Q,not very helpful on the phone. changed their p...,3.0


In [ ]:
print(len(test_reviews.index), "yelp reviews for testing")
test_reviews

53359 yelp reviews for testing


,review_id,text,stars
138379,pfwXHRWq2jzT00n2BCbUnQ,I would avoid doing business with Nationwide a...,1.0
382311,7rMAnoL5AQbU2PCjH_4hvg,Why is it so hard to find a jamba juice in veg...,4.0
93399,W9WwTpqqOmHaEkDjFGY3kg,Definetly a fine ice cream chain. The massive ...,4.0
122224,PNYbsSoEvoeVHhrlaBYxkw,One of a very few places in gta where one can ...,5.0
224622,TDl1VLEcC49LMfhucNRDZw,Was told I'm a pain in the ass customer. Dana ...,1.0
...,...,...,...
359783,V89YHw66stC6m2q_Q8f1QA,First Yelp review I've ever written. They were...,1.0
152315,PkW-fQcMLCYK1EXXlbghxA,Excellent! I always seem to get the huevos ran...,4.0
117952,P1-rGkBlw_PHDAz2pNd4pA,"Sorry Mitchell's, I'm a Graeter's girl now. I ...",5.0
435829,rE4xHmpnFp6xaWJRWsGSvA,Jay's was referred to me by my neighbor who is...,5.0


#Model

## model construction

In [ ]:
model = LanguageModel(vocab_size=MAX_LEN, rnn_size=256, vader_size=MAX_LEN_VADER)

TypeError: ignored

## train the model

In [ ]:
# set model to training mode
model.train()

In [ ]:
# https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html

# TODO: fix this block
DATASET = train_reviews


since = time.time()

# start training
for epoch in range(num_epochs):
  indices = np.random.permutation(range(DATASET.size))
  t = tqdm.notebook.tqdm(range(0, ( DATASET.size //batch_size)+1))
  
  for i in t:
    # batch
    batch = format_reviews(xlnet_tokenizer, DATASET, indices[i*batch_size:(i+1)*batch_size])
    (batch_input, batch_target, batch_target_mask) = batch_to_torch(*batch)
    for item in (batch_input, batch_target, batch_target_mask):
      print(item.size())
    (batch_input, batch_target, batch_target_mask) = list_to_device((batch_input, batch_target, batch_target_mask))
    
    # forward pass
    prediction = model.forward(batch_input)
    loss = loss_fn(prediction, batch_target, batch_target_mask)
    losses.append(loss.item())
    accuracy = (th.eq(prediction.argmax(dim=2,keepdim=False),batch_target).float()*batch_target_mask).sum()/batch_target_mask.sum()
    accuracies.append(accuracy.item())
    
    # backward pass
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    # visuallize data
    if i % 100 == 0:
        batch_val = build_batch(d_valid, range(len(d_valid)))
        (batch_input_val, batch_target_val, batch_target_mask_val) = list_to_device(batch_to_torch(*batch_val))
        prediction_val = model.forward(batch_input_val)
        loss_val = loss_fn(prediction_val, batch_target_val, batch_target_mask_val)
        print("Evaluation set loss:", loss_val.item())
        print(f"Epoch: {epoch} Iteration: {i} Loss: {np.mean(losses[-10:])} Accuracy: {np.mean(accuracies[-10:])}")



## evaluate model

In [ ]:
# set model to evaluation model
model.eval()

In [ ]:
# model(yelp_ratings["text"])

#Playground

In [ ]:
# This is a cheap solution to stops any run all that reaches my Playground
hard_stop = input("Hard Stop here. Enter any key to allow passage.")

if len(hard_stop) == 0:
  raise Exception("Hard Stop")

In [ ]:
print(list(tokenize(STARTER["text"]))[:3])

In [ ]:
import urllib.request
import io
import sentencepiece as spm

# https://github.com/google/sentencepiece/tree/master/python

# Loads model from URL as iterator and stores the model to BytesIO.
model = io.BytesIO()
spm.SentencePieceTrainer.train(
      sentence_iterator=STARTER["text"], model_writer=model, vocab_size=1000)

# Serialize the model as file.
# with open('out.model', 'wb') as f:
#   f.write(model.getvalue())

# Directly load the model from serialized model.
sp = spm.SentencePieceProcessor(model_proto=model.getvalue())
print(sp.encode('this is test'))